In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

# 1. Load and prepare data
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/NLP/Tweets.csv")
df = df[["text", "airline_sentiment"]]

In [3]:
le = LabelEncoder()
df['label'] = le.fit_transform(df['airline_sentiment'])

dataset = Dataset.from_pandas(df[['text','label']])
dataset = dataset.train_test_split(test_size=0.2)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [9]:
def tokenize_function(tweets):
    return tokenizer(tweets['text'], padding='max_length', truncation=True, max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

#pretrained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=3)
training_args = TrainingArguments(
    output_dir='./results',
    #evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
)

Map:   0%|          | 0/11712 [00:00<?, ? examples/s]

Map:   0%|          | 0/2928 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
from sklearn.metrics import accuracy_score
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


/tmp/ipython-input-10-4073061578.py:8: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [11]:
trainer.train()

Step,Training Loss
500,0.546900
1000,0.357300
1500,0.291000
2000,0.200600


TrainOutput(global_step=2196, training_loss=0.33398887933060556, metrics={'train_runtime': 972.6759, 'train_samples_per_second': 36.123, 'train_steps_per_second': 2.258, 'total_flos': 2311188261322752.0, 'train_loss': 0.33398887933060556, 'epoch': 3.0})

In [12]:
trainer.evaluate()

{'eval_loss': 0.4731791317462921,
 'eval_accuracy': 0.8596311475409836,
 'eval_runtime': 19.9593,
 'eval_samples_per_second': 146.699,
 'eval_steps_per_second': 9.169,
 'epoch': 3.0}

In [13]:
model.save_pretrained('./sentiment_bert')
tokenizer.save_pretrained('./sentiment_bert')

('./sentiment_bert/tokenizer_config.json',
 './sentiment_bert/special_tokens_map.json',
 './sentiment_bert/vocab.txt',
 './sentiment_bert/added_tokens.json')